# A/B Testing Analysis with Superset

This notebook provides SQL queries and chart configurations for analyzing the A/B testing simulation data in Apache Superset.

## Generated Test Data Overview

We've created 4 different A/B tests with the following scenarios:

1. **App Icon Design Test**: Control vs Minimalist vs Colorful icons
2. **App Description Length Test**: Short vs Detailed descriptions
3. **Pricing Strategy Test**: Free vs Freemium vs Paid
4. **Screenshot Count Test**: 3 vs 5 screenshots

**Data Generated**: 15,000 user sessions over 45 days

## Database Tables Created

### 1. `ab_test_sessions` (Main data table)
Contains detailed user session data with:
- User demographics (age_group, device_type, country)
- App information (app_name, app_genre)
- Test variant assignments
- Behavioral metrics (time_spent, converted, rating)

### 2. `ab_test_variants` (Test definitions)
Contains test variant definitions and descriptions

### 3. `ab_test_summary` (Pre-aggregated results)
Contains aggregated conversion rates and metrics by variant

## Setting up Superset

### Step 1: Access Superset
1. Open your browser and go to: http://localhost:8088
2. Login with:
   - Username: `admin`
   - Password: `admin`

### Step 2: Add Database Connection
1. Go to **Settings** → **Database Connections**
2. Click **+ Database**
3. Select **PostgreSQL**
4. Use these connection details:
   ```
   Host: db (internal Docker network)
   Port: 5432
   Database: analytics_db
   Username: postgres
   Password: postgres
   ```
5. Test connection and save

## Recommended Charts for Superset

### Chart 1: A/B Test Conversion Rates Comparison
**Chart Type**: Bar Chart
**Table**: ab_test_summary

In [ ]:
-- SQL Query for Conversion Rates Comparison
SELECT 
    test_name,
    variant_key,
    total_users,
    conversions,
    ROUND(conversion_rate * 100, 2) as conversion_rate_percent
FROM ab_test_summary
ORDER BY test_name, conversion_rate DESC;

**Chart Configuration**:
- **X-axis**: test_name + variant_key (concatenated)
- **Y-axis**: conversion_rate_percent
- **Color**: test_name
- **Sort**: By conversion_rate DESC

### Chart 2: Daily Conversion Trends
**Chart Type**: Line Chart
**Table**: ab_test_sessions

In [ ]:
-- SQL Query for Daily Conversion Trends
SELECT 
    DATE(session_date) as session_day,
    icon_design_variant,
    COUNT(*) as total_sessions,
    SUM(CASE WHEN converted THEN 1 ELSE 0 END) as conversions,
    ROUND(AVG(CASE WHEN converted THEN 1.0 ELSE 0.0 END) * 100, 2) as conversion_rate
FROM ab_test_sessions
GROUP BY DATE(session_date), icon_design_variant
ORDER BY session_day, icon_design_variant;

**Chart Configuration**:
- **X-axis**: session_day
- **Y-axis**: conversion_rate
- **Series**: icon_design_variant
- **Time Grain**: Day

### Chart 3: Conversion Rate by Demographics
**Chart Type**: Heatmap
**Table**: ab_test_sessions

In [ ]:
-- SQL Query for Demographics Heatmap
SELECT 
    age_group,
    device_type,
    COUNT(*) as total_users,
    ROUND(AVG(CASE WHEN converted THEN 1.0 ELSE 0.0 END) * 100, 1) as conversion_rate
FROM ab_test_sessions
GROUP BY age_group, device_type
HAVING COUNT(*) > 50
ORDER BY age_group, device_type;

**Chart Configuration**:
- **X-axis**: device_type
- **Y-axis**: age_group
- **Metric**: conversion_rate
- **Color Scheme**: Red-Yellow-Blue

### Chart 4: Time Spent Analysis by Variant
**Chart Type**: Box Plot
**Table**: ab_test_sessions

In [ ]:
-- SQL Query for Time Spent Analysis
SELECT 
    description_variant,
    screenshots_variant,
    time_spent_seconds,
    converted
FROM ab_test_sessions
WHERE time_spent_seconds BETWEEN 5 AND 300  -- Filter outliers
ORDER BY description_variant, screenshots_variant;

**Chart Configuration**:
- **X-axis**: description_variant + screenshots_variant
- **Y-axis**: time_spent_seconds
- **Series**: converted (True/False)

### Chart 5: Geographic Performance
**Chart Type**: World Map or Bar Chart
**Table**: ab_test_sessions

In [ ]:
-- SQL Query for Geographic Analysis
SELECT 
    country,
    pricing_variant,
    COUNT(*) as total_users,
    SUM(CASE WHEN converted THEN 1 ELSE 0 END) as conversions,
    ROUND(AVG(CASE WHEN converted THEN 1.0 ELSE 0.0 END) * 100, 2) as conversion_rate,
    ROUND(AVG(time_spent_seconds), 1) as avg_time_spent
FROM ab_test_sessions
GROUP BY country, pricing_variant
HAVING COUNT(*) > 20
ORDER BY country, conversion_rate DESC;

**Chart Configuration**:
- **X-axis**: country
- **Y-axis**: conversion_rate
- **Series**: pricing_variant
- **Secondary Y-axis**: avg_time_spent

### Chart 6: App Genre Performance
**Chart Type**: Treemap
**Table**: ab_test_sessions

In [ ]:
-- SQL Query for App Genre Analysis
SELECT 
    app_genre,
    icon_design_variant,
    COUNT(*) as total_sessions,
    ROUND(AVG(CASE WHEN converted THEN 1.0 ELSE 0.0 END) * 100, 2) as conversion_rate,
    ROUND(AVG(CASE WHEN rating IS NOT NULL THEN rating ELSE NULL END), 2) as avg_rating
FROM ab_test_sessions
GROUP BY app_genre, icon_design_variant
HAVING COUNT(*) > 30
ORDER BY app_genre, conversion_rate DESC;

**Chart Configuration**:
- **Primary Dimension**: app_genre
- **Secondary Dimension**: icon_design_variant
- **Size Metric**: total_sessions
- **Color Metric**: conversion_rate

### Chart 7: Statistical Significance Test
**Chart Type**: Table
**Table**: Custom Query

In [ ]:
-- SQL Query for Statistical Significance Analysis
WITH test_stats AS (
    SELECT 
        'Icon Design Test' as test_name,
        icon_design_variant as variant,
        COUNT(*) as sample_size,
        SUM(CASE WHEN converted THEN 1 ELSE 0 END) as conversions,
        AVG(CASE WHEN converted THEN 1.0 ELSE 0.0 END) as conversion_rate
    FROM ab_test_sessions
    GROUP BY icon_design_variant
    
    UNION ALL
    
    SELECT 
        'Description Test' as test_name,
        description_variant as variant,
        COUNT(*) as sample_size,
        SUM(CASE WHEN converted THEN 1 ELSE 0 END) as conversions,
        AVG(CASE WHEN converted THEN 1.0 ELSE 0.0 END) as conversion_rate
    FROM ab_test_sessions
    GROUP BY description_variant
    
    UNION ALL
    
    SELECT 
        'Pricing Test' as test_name,
        pricing_variant as variant,
        COUNT(*) as sample_size,
        SUM(CASE WHEN converted THEN 1 ELSE 0 END) as conversions,
        AVG(CASE WHEN converted THEN 1.0 ELSE 0.0 END) as conversion_rate
    FROM ab_test_sessions
    GROUP BY pricing_variant
)
SELECT 
    test_name,
    variant,
    sample_size,
    conversions,
    ROUND(conversion_rate * 100, 2) as conversion_rate_percent,
    CASE 
        WHEN sample_size > 1000 AND conversion_rate > 0.05 THEN 'Statistically Significant'
        WHEN sample_size > 500 THEN 'Moderate Significance'
        ELSE 'Insufficient Data'
    END as significance_level
FROM test_stats
ORDER BY test_name, conversion_rate DESC;

## Dashboard Creation Guide

### Step 1: Create Individual Charts
1. Go to **Charts** in Superset
2. Click **+ Chart**
3. Select your PostgreSQL database connection
4. Choose the appropriate table
5. Copy and paste the SQL queries above
6. Configure chart settings as described
7. Save each chart with descriptive names

### Step 2: Create Dashboard
1. Go to **Dashboards**
2. Click **+ Dashboard**
3. Name it "A/B Testing Analysis Dashboard"
4. Add all your created charts
5. Arrange them in a logical layout

### Step 3: Add Filters
Add dashboard-level filters for:
- Date range (session_date)
- App genre
- Country
- Device type
- Age group

## Key Insights to Look For

Based on the simulation, you should expect to see:

1. **Icon Design Test**: Colorful icons (variant_b) should perform best
2. **Description Test**: Detailed descriptions (variant_a) should have higher conversion
3. **Pricing Test**: Free apps (control) should have highest conversion rates
4. **Screenshots Test**: More screenshots (variant_a) should perform slightly better

### Additional Analysis Questions:
- Which age groups respond best to different variants?
- Do conversion patterns vary by device type?
- Are there geographic differences in variant performance?
- How does time spent correlate with conversion likelihood?
- Do different app genres respond differently to the same variants?

## Next Steps

1. **Create the charts in Superset** using the SQL queries above
2. **Build the dashboard** with all charts arranged logically
3. **Add interactivity** with filters and drill-down capabilities
4. **Share insights** with stakeholders
5. **Iterate and improve** based on findings

Remember: This is simulated data, but the analysis techniques and chart types are exactly what you'd use for real A/B testing data!